In [1]:
# Di dalam notebook Anda
import pandas as pd
import random

# Kamus entitas
lokasi = ["di Jalan Sudirman", "di Mall Grand Indonesia", "dekat Monas", "di Apartemen Green Pramuka", "di Stasiun Manggarai", "kawasan SCBD"]
jenis_kebakaran = ["kebakaran hebat", "asap tebal", "ledakan tabung gas", "korsleting listrik"]
jenis_medis = ["serangan jantung", "kecelakaan motor", "pingsan tidak sadarkan diri", "pendarahan hebat", "kesulitan bernapas"]
jenis_kriminal = ["pencurian dengan kekerasan", "perampokan toko", "penjambretan", "tawuran warga"]
detail_objek = ["rumah tinggal", "gedung perkantoran", "mobil sedan silver", "warung kelontong"]

# Template kalimat berdasarkan kategori
templates = {
    "Kebakaran": [
        "Tolong, ada {jenis_kebakaran} di {lokasi}, korbannya di dalam {detail_objek}.",
        "Lapor! Terjadi {jenis_kebakaran} {lokasi}.",
        "Darurat! Api melalap {detail_objek} {lokasi}."
    ],
    "Medis": [
        "Cepat, butuh ambulans! Ada korban {jenis_medis} {lokasi}.",
        "Seseorang butuh pertolongan medis karena {jenis_medis} {lokasi}.",
        "Tolong, ada yang {jenis_medis} di {detail_objek} {lokasi}."
    ],
    "Kriminal": [
        "Waspada! Terjadi {jenis_kriminal} {lokasi}.",
        "Saya baru saja melihat {jenis_kriminal} di {detail_objek} {lokasi}, pelakunya kabur.",
        "Lapor, ada insiden {jenis_kriminal} {lokasi}."
    ]
}

In [2]:
# Lanjutan di notebook
data = []
for i in range(500): # Hasilkan 500 sampel
    category = random.choice(list(templates.keys()))
    template = random.choice(templates[category])
    
    # Pilih entitas secara acak
    subs = {
        "lokasi": random.choice(lokasi),
        "jenis_kebakaran": random.choice(jenis_kebakaran),
        "jenis_medis": random.choice(jenis_medis),
        "jenis_kriminal": random.choice(jenis_kriminal),
        "detail_objek": random.choice(detail_objek),
    }
    
    # Format kalimat
    text = template.format(**subs)
    
    data.append({"text": text, "category": category})

df = pd.DataFrame(data)
df.to_csv('../data/processed/synthetic_data.csv', index=False)
print(df.head())

                                                text  category
0  Lapor, ada insiden perampokan toko di Stasiun ...  Kriminal
1  Tolong, ada yang kecelakaan motor di gedung pe...     Medis
2  Tolong, ada yang kecelakaan motor di rumah tin...     Medis
3        Waspada! Terjadi penjambretan kawasan SCBD.  Kriminal
4  Seseorang butuh pertolongan medis karena penda...     Medis


In [8]:
import os
import pandas as pd
from gtts import gTTS
import ffmpeg  # <-- Gunakan library baru

# Pastikan variabel 'df' sudah ada dari sel-sel sebelumnya
# df = pd.read_csv('../data/processed/synthetic_data.csv')

# Pastikan direktori output ada
output_dir = '../data/raw_audio/'
os.makedirs(output_dir, exist_ok=True)

print("Memulai pembuatan file audio menggunakan gTTS dan ffmpeg...")

# --- PENTING: Hapus file audio lama sebelum memulai ---
for f in os.listdir(output_dir):
    os.remove(os.path.join(output_dir, f))
print("File audio lama telah dihapus.")
# ----------------------------------------------------

for index, row in df.iterrows():
    temp_filename = f"{output_dir}temp_{index}.mp3"
    final_filename = f"{output_dir}call_{index}.wav"
    
    # 1. Buat file MP3 sementara menggunakan gTTS
    tts = gTTS(row['text'], lang='id')
    tts.save(temp_filename)
    
    # 2. Gunakan ffmpeg untuk mengkonversi MP3 ke WAV 16kHz Mono
    try:
        (
            ffmpeg
            .input(temp_filename)
            .output(
                final_filename,
                acodec='pcm_s16le',  # Format standar untuk WAV (sama dengan LINEAR16)
                ac=1,               # Audio Channels = 1 (Mono)
                ar='16000'          # Audio Rate = 16000 Hz
            )
            .run(overwrite_output=True, quiet=True) # quiet=True agar tidak banyak log
        )
    except ffmpeg.Error as e:
        print(f"Gagal mengkonversi file {temp_filename}: {e.stderr}")

    # 3. Hapus file MP3 sementara
    os.remove(temp_filename)
    
    if (index + 1) % 50 == 0:
        print(f"Selesai membuat {index + 1}/{len(df)} file audio.")

print(f"SELESAI! {len(df)} file audio telah dibuat di folder {output_dir}")

Memulai pembuatan file audio menggunakan gTTS dan ffmpeg...
File audio lama telah dihapus.
Selesai membuat 50/500 file audio.
Selesai membuat 100/500 file audio.
Selesai membuat 150/500 file audio.
Selesai membuat 200/500 file audio.
Selesai membuat 250/500 file audio.
Selesai membuat 300/500 file audio.
Selesai membuat 350/500 file audio.
Selesai membuat 400/500 file audio.
Selesai membuat 450/500 file audio.
Selesai membuat 500/500 file audio.
SELESAI! 500 file audio telah dibuat di folder ../data/raw_audio/
